In [1]:
import yaml
from pathlib import Path
from utils import get_data_loaders, load_checkpoint, log_epoch, Criterion, add_cuts_to_config
import torch
from models import Model
from tqdm import tqdm
import pandas as pd

In [3]:
cuda_id = 3
log_name = '05_16_2022_15_49_29-GNN_full_dR_1'  # log id of the saved model to load
setting = log_name.split('-')[1]

config = yaml.safe_load(Path(f'./configs/{setting}.yml').open('r'))
device = torch.device(f'cuda:{cuda_id}' if cuda_id >= 0 else 'cpu')
log_path = Path(config['data']['log_dir']) / log_name

if len(log_name.split('-')) > 2:
    cut_id = log_name.split('-')[2]
    config = add_cuts_to_config(config, cut_id)

In [4]:
data_loaders, x_dim, edge_attr_dim, dataset = get_data_loaders(setting, config['data'], config['optimizer']['batch_size'])


[Splits]
    train: 332406. # pos: 55401, # neg: 277005. Pos:Neg: 0.200
    valid: 71226. # pos: 11871, # neg: 59355. Pos:Neg: 0.200
    test: 175113. # pos: 11873, # neg: 163240. Pos:Neg: 0.073


In [5]:
df = pd.read_pickle(dataset.get_df_save_path())
assert len(df) == len(dataset)  # let's not consider the half detector case, where len(dataset) will be roughly doubled as each neg200 gives 2 neg samples (both endcaps)

In [6]:
model = Model(x_dim, edge_attr_dim, config['data']['virtual_node'], config['model']).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=config['optimizer']['lr'])
load_checkpoint(model, optimizer, log_path, device)
criterion = Criterion(config['optimizer'])

[INFO] Loading checkpoint from 05_16_2022_15_49_29-GNN_full_dR_1


In [7]:
@torch.no_grad()
def eval_one_batch(data, model, criterion):
    model.eval()

    clf_logits = model(x=data.x, edge_index=data.edge_index, edge_attr=data.edge_attr, batch=data.batch, data=data)
    loss, loss_dict = criterion(clf_logits.sigmoid(), data.y)
    return loss_dict, clf_logits.data.cpu()


def run_one_epoch(data_loader, epoch, phase, device, model, criterion):
    loader_len = len(data_loader)
    run_one_batch = eval_one_batch
    phase = 'test ' if phase == 'test' else phase  # align tqdm desc bar

    all_loss_dict, all_clf_logits, all_clf_labels, all_sample_idx = {}, [], [], []
    pbar = tqdm(data_loader, total=loader_len)
    for idx, data in enumerate(pbar):
        loss_dict, clf_logits = run_one_batch(data.to(device), model, criterion)

        desc = log_epoch(epoch, phase, loss_dict, clf_logits, data.y.data.cpu(), batch=True)
        for k, v in loss_dict.items():
            all_loss_dict[k] = all_loss_dict.get(k, 0) + v
        all_clf_logits.append(clf_logits), all_clf_labels.append(data.y.data.cpu()), all_sample_idx.append(data.sample_idx.data.cpu())

        if idx == loader_len - 1:
            all_clf_logits, all_clf_labels, all_sample_idx = torch.cat(all_clf_logits), torch.cat(all_clf_labels), torch.cat(all_sample_idx)
            for k, v in all_loss_dict.items():
                all_loss_dict[k] = v / loader_len
            desc, auroc, recall, avg_loss = log_epoch(epoch, phase, all_loss_dict, all_clf_logits, all_clf_labels, False, None)
        pbar.set_description(desc)

    return avg_loss, auroc, recall, all_clf_logits, all_sample_idx

In [8]:
clf_probs, all_sample_idx = [], []
for phase in ['train', 'valid', 'test']:
    avg_loss, auroc, recall, clf_logits, sample_idx = run_one_epoch(data_loaders[phase], 999, phase, device, model, criterion)
    clf_probs.append(clf_logits.sigmoid())
    all_sample_idx.append(sample_idx)
clf_probs = torch.cat(clf_probs)
all_sample_idx = torch.cat(all_sample_idx)

[Epoch: 999]: train finished, focal: 0.006, total: 0.006, auroc: 0.636, recall@maxfpr: 0.003: 100%|██████████| 1299/1299 [01:12<00:00, 17.98it/s]
[Epoch: 999]: valid finished, focal: 0.006, total: 0.006, auroc: 0.636, recall@maxfpr: 0.002: 100%|██████████| 279/279 [00:15<00:00, 18.30it/s]
[Epoch: 999]: test  finished, focal: 0.005, total: 0.005, auroc: 0.639, recall@maxfpr: 0.003: 100%|██████████| 685/685 [00:38<00:00, 17.60it/s]


In [9]:
scores = pd.DataFrame({'sample_idx': all_sample_idx, 'probs': clf_probs.reshape(-1)})
scores = scores.sort_values('sample_idx').reset_index(drop=True)

In [20]:
scores.to_pickle(dataset.get_df_save_path().parent / f'{setting}-scores.pkl')

In [10]:
scores

,sample_idx,probs
0,0,0.509518
1,1,0.506023
2,2,0.523474
3,3,0.550908
4,4,0.528242
...,...,...
578740,578740,0.469370
578741,578741,0.470245
578742,578742,0.490662
578743,578743,0.523394
